In [1]:
import pandas as pd
from util import *

In [2]:
stationNameCode = loadPickle("../trimmed_data/stationNameCode.pickel")
( type(stationNameCode), stationNameCode["경성대.부경대"] )

(dict, 212)

In [3]:
HJDONG_AREA_GIS = pd.read_csv("../trimmed_data/HJDONG_AREA_GIS.csv")
HJDONG_AREA_GIS.head()

,LineCode,StationName,HJDONG_NAME,HJDONG_AREA,NEAR_STATION_AREA
0,1,동매,부산광역시 사하구 괴정1동,1138082.148,36465.620
1,1,동매,부산광역시 사하구 괴정4동,866771.804,158372.769
2,1,동매,부산광역시 사하구 당리동,3927869.185,96049.312
3,1,동매,부산광역시 사하구 하단1동,8258339.734,4562.966
4,1,동매,부산광역시 사하구 신평1동,1765292.312,1765292.312


In [4]:
FLUX_METRIC = pd.read_csv("../trimmed_data/FLUX_METRIC.csv")
FLUX_METRIC.head()

,HJDONG,직장인구,주거인구,유동인구
0,대저1동,13999.0,5912.0,142306.0
1,대저2동,22963.0,6615.0,138681.0
2,강동동,3346.0,3892.0,60696.0
3,명지1동,14891.0,54365.0,159545.0
4,명지2동,7542.0,30168.0,236941.0
